In [65]:
import time
import string
import requests
import re

In [1]:
from selenium import webdriver
driver = webdriver.Firefox()

# Scraping all work urls from main personal page 

In [42]:
URL = "http://changba.com/u/26936044"
driver.get(URL)
while True:
    try: 
        driver.find_element_by_id("loadWork").click()
    except: 
        break
    time.sleep(2)
#the main page only shows your most recent works which is only part of the whole list
#we need to keep clicking "加載更多" (load more) until we no more available 

In [43]:
# if we only want the name list:
work_list=driver.find_element_by_id('work_list')
len(work_list.text.split('\n'))

837

In [66]:
# get the list of name and corresponding url
WorkUrl_list=[]
works=driver.find_elements_by_class_name('userPage-work-li') # all work nodes belong to this class
#you can also use clearfix which actually works better since the last one does not belongs to this class
for work in works:
    try: 
        child=work.find_element_by_xpath(".//*")#By this step i can find the only child (attribute node) with attributes <a href=...></a>
    except: 
        break #the last one is blank with no child
    name=string.capwords(child.text.split('【')[0].lower())#Name Format
    url=child.get_attribute('href')
    WorkUrl_list.append((name,url)) # save a list of tuples [(work name, url)]

忘我
Real Friends
When I Was Your Man
Nasa
Scars To Your
起风了
Somethings Got A Hold On Me
身骑白马
天天想你
Hollywood
Cry Me A River
Stay
One Kiss
Final Song
Beautiful
Shut Up And Drive
Gimme More
Dance Monkey
All I Want For Christmas Is You
摩天动物园
摩天动物园
红玫瑰
魅惑
Living Proof
Cry For Me
红眼睛
Desperado
舞娘
特务j
Forever Friends
A Little Love
我比从前更寂寞
Never Ending
线的记忆
Trampoline
微光
一个人想你
风从海面吹过来
同步
路痴
Diamonds
鱼仔
Facetime.moli
陪你玩到底.moli
你敢我就敢
20.moli
曾经守候
不亏不欠
Often
Say You Won't Let Go
Paper Hearts
Circles
莱特兄弟有罪
Love Galore
Forever Young
给朱利安
给朱利安
Spirit
Slide Away
Someone You Loved
就是现在
就是现在
甜秘密
风起云涌
别再问我什么是迪斯科
Idontwannabeyouanymore
Somebody
On A Roll
玫瑰少年
Goodnight N Go
Señorita
Señorita
Thunderclouds
沉船
让她降落
雪落下的声音
Don't Leave Me Lonely
一个人没有同类
爱
My Songs Know What You Did In The Dark
逆光
光荣
梦里花
Streetcar
Do You Remember
凤凰于飞
Breathless
Glamorous
很久以前
Extraordinary Being
Disturbia
Best Part
彩虹
无言
白月光
那个男人
我愿意
Peace
Stay
Wish You Were Gay
I Love You
Dusk Till Dawn
Said Too Much
好肉
Try
Move Your Body


In [60]:
len(WorkUrl_list)

837

In [ ]:
#for work in works:
#    url=driver.find_element_by_link_text(work.text).get_attribute('href')
#    WorkUrl_list.append((work.text,url))

In [261]:
import pandas as pd 
pd_pages = pd.DataFrame(WorkUrl_list, columns = ['name', 'Page'])

# Scraping a page for single work

In [262]:
pd_pages.loc[3]

name                                           Nasa
Page    http://changba.com/s/GZr_J5_BvVpGx_odmAqtBQ
Name: 3, dtype: object

In [264]:
#scrap some info about this work
URL = pd_pages.loc[3].Page # get url
driver.get(URL) # get page 
audience=driver.find_element_by_class_name('audience').text # get information in audience node
comment=driver.find_element_by_class_name('comment').text
presents=driver.find_element_by_class_name('presents').text
share=driver.find_element_by_class_name('share').text

Bad new is there is no time info available on the website (unlike phone). The only way we can estimate the submitting time is to use the earliest comment posting time. 

In [203]:
#estimate submitting time
times=driver.find_elements_by_class_name('post-time') #get post-time of comments
if len(times) >0: # if there is comments
    time=pd.Timestamp(times[-1].text) # take the earliest time    

In [233]:
#locate and scrap the url of this work
html=str(requests.get(URL).content)
start, end =re.search(r'http://upscuw.changba.com/\d*\.mp3',html).span() # find mp3 file url in raw html
song_url=html[start:end]

# For all pages

In [347]:
from selenium.common.exceptions import NoSuchElementException
#it happens when the page is forbidden

In [336]:
pd_urls=pd.DataFrame(None,
                     columns = ['audience', 'comment','presents','share','url','Time'],
                     index=pd_pages.index)

In [390]:
for i in pd_urls.index:
    URL = pd_pages.loc[i].Page
    try:
        driver.get(URL)
        pd_urls.audience[i]=driver.find_element_by_class_name('audience').text
        pd_urls.comment[i]=driver.find_element_by_class_name('comment').text
        pd_urls.presents[i]=driver.find_element_by_class_name('presents').text
        pd_urls.share[i]=driver.find_element_by_class_name('share').text
        
        times=driver.find_elements_by_class_name('post-time') #get post-time of comments
        if len(times) >0: # if there is comments
            pd_urls.Time[i]=pd.Timestamp(times[-1].text) # take the earliest time
        
        html=str(requests.get(URL).content)
        start, end =re.search(r'a="https*://.*\.changba\.com.*/\d*\.mp3',html).span()
        pd_urls.url[i]=html[start+3:end] # updated from previous one to be more robost.
    
    except NoSuchElementException: # the work is not permited to access due to sensitive words in name
        print('NoSuchElementException for index ',i)
        continue
    except AttributeError: # no .mp3 file found in html (if there is a mv)
        print('AttributeError for index ',i)
        continue
    except KeyboardInterrupt: # control to stop 
        break
    time.sleep(2) # avoid frequent request (previous steps are slow tho)

Xo
AttributeError for index  157
山楂树
Ur So Gay
Genius
Count On Me
解脱
剪爱
Imagine
Thinking Bout You
Ghostin
Break Up With Your Girlfriend
Needy
Bad Idea
How Deep Is Your Love
Apeshit
NoSuchElementException for index  171
Goodnight N Go
Forever Young
让每个人都心碎
Give Me Love
杀破狼
时候
起风了
7 Rings
NoSuchElementException for index  179
7 Rings
Dancing With A Stranger
阴天快乐
飘向北方
空港
Marry You
Just The Way You Are
千万次的问
天地在我心
忽然之间
写一首歌
等待
一呼...一吸
偷故事的人
魅力无限
Time Decides
Perfect
Rehab
Stupid In Love
袖手旁观
Lost Boy
Lost Boy
日子
Big Girls Cry
我好想你
Bird Set Free
Alive
高明的悲剧
剑雨浮生
December
天下无双
煎熬
左手指月
左手指月
幸福快车
沙文
归零
幻觉动物
纤维
Consequences
Royals
不正确的审美 (cindy Solo)
我们的爱
遗憾
遗书
蜜蜂
我们在蓝色海上飘
我
Stitches
Thank U Next
你是我的眼
我离开我自己
莫名其妙爱上你
Work Bitch
NoSuchElementException for index  233
Freakshow(带和声）
Everytime
微光
证据
Partition
All Night
仰望星空
Heartless
小宇宙
Call Out My Name
之外
别闹
Angel Down
双影
光年之外
Breathin
半途而废
God Is A Woman
盛夏的果实
Prisoner
I Feel It Coming
Home Sweet Home
好久不见
需要人陪
晚风
9277
可惜不是你
魔鬼中的天使
天真有邪
不睡
连名带姓


In [420]:
driver.close()
pd_urls.to_csv('changba_urls.csv')

In [551]:
pd_merge=pd_pages.join(pd_urls)
pd_merge

,name,Page,audience,comment,presents,share,url,Time
0,忘我,http://changba.com/s/EXb4wphbN-GY464jDRV_KA,64,4,8,0,http://upuwmp3.changba.com/userdata/userwork/9...,2019-12-29 00:00:00
1,Real Friends,http://changba.com/s/FBZMCWgrLY3sO7sUEFbgSg,32,0,5,0,http://ksapuw.changba.com/userdata/userwork/81...,NaN
2,When I Was Your Man,http://changba.com/s/GZr_J5_BvVr4a-ftEiASmw,29,3,4,2,http://upscuw.changba.com/1208653739.mp3,2019-12-29 00:00:00
3,Nasa,http://changba.com/s/GZr_J5_BvVpGx_odmAqtBQ,25,0,3,0,http://upscuw.changba.com/1208653711.mp3,NaN
4,Scars To Your,http://changba.com/s/oTPTKN_3qXjKpIfapWDAAA,16,0,0,0,http://qiniuuwmp3.changba.com/1208653567.mp3,NaN
...,...,...,...,...,...,...,...,...
832,恒星流星,http://changba.com/s/lxnKkg0S0RVrwOn6rjxZOg,41,0,0,0,http://upuwmp3.changba.com/userdata/userwork/8...,2013-08-05 00:00:00
833,也许明天,http://changba.com/s/z1FbhFD6pZ669cvjWIcqfA,106,0,4,0,http://upuwmp3.changba.com/userdata/userwork/8...,NaN
834,剪爱,http://changba.com/s/TIO4hN908E8Wndka1ntQpQ,26,0,0,0,http://upuwmp3.changba.com/userdata/userwork/7...,2013-08-01 00:00:00
835,空白格,http://changba.com/s/co-YKmeP2LT5THi79AR9gQ,72,0,3,0,http://upuwmp3.changba.com/userdata/userwork/5...,2013-08-15 00:00:00


## One interesting question
Recall that we estimate the submitting time by earliest comment time, some work has **no comment** and some work are firstly commented **later than submitted**. Hence,the time stamps are collected with some **missing time** and some **delayed time** and we want to correct them such that our time stamps should be consistent in **non-increasing order**. How to correct this time series?

Denote the timesatamps as $t_i$ and our estimate as $\hat{t}_i$, what we are sure about is:
+ $t_{i+1}\ge t_i$
+ $\hat{t}_i\ge t_i$   if $\hat{t}_i$ is not missing
+ we assume that most timestamp estimates including the first and the last one are correct.

How can we check if a timestamp series is non-increasing?

A: it is non-increasing everywhere. any adjacent pais is non-increasing.

In [535]:
def check_non_increasing(times):
    increase=0
    for i in range(len(times)-1):
        now=i
        before=i+1
        if times[now]<times[before]: # there is a increase.
            increase+=1 # you can also stop and return something here
    return increase

There are two extreme ways of correction:

1. get the upper bound of corrected time

for each correction, let it be as large as possible，向前看齊

In [552]:
pd_merge['CorrectedTime_U']=None
earliest_time=pd_merge.Time[0]
for i in pd_merge.index:
    if (not pd.isna(pd_merge.Time[i]) and pd_merge.Time[i] <= earliest_time):
        earliest_time = pd_merge.Time[i]
    pd_merge.CorrectedTime_U[i] = earliest_time

2. get the lower bound of corrected time

for each correction, let it be as small as possible, 向後看齊

In [553]:
pd_merge['CorrectedTime_L']=None
pd_merge['fault']=False
earliest_time=pd_merge.Time[0]# the latest time observed so far
for i in pd_merge.index:
    if pd.isna(pd_merge.Time[i]) or pd_merge.Time[i] > earliest_time:
        pd_merge.fault[i]=True
    else:
        earliest_time = pd_merge.Time[i]

D:\Anoconda\envs\Python\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [554]:
latest_time=pd_merge.Time[836]# the latest time observed so far
for i in range(836,-1,-1):
    if not pd_merge.fault[i]:
        latest_time = pd_merge.Time[i]
    pd_merge.CorrectedTime_L[i]=latest_time    

D:\Anoconda\envs\Python\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


check if there is still faults

In [555]:
check_non_increasing(pd_merge.CorrectedTime_L)

0

In [556]:
check_non_increasing(pd_merge.CorrectedTime_U)

0

In [557]:
pd_merge

,name,Page,audience,comment,presents,share,url,Time,CorrectedTime_U,CorrectedTime_L,fault
0,忘我,http://changba.com/s/EXb4wphbN-GY464jDRV_KA,64,4,8,0,http://upuwmp3.changba.com/userdata/userwork/9...,2019-12-29 00:00:00,2019-12-29 00:00:00,2019-12-29 00:00:00,False
1,Real Friends,http://changba.com/s/FBZMCWgrLY3sO7sUEFbgSg,32,0,5,0,http://ksapuw.changba.com/userdata/userwork/81...,NaN,2019-12-29 00:00:00,2019-12-29 00:00:00,True
2,When I Was Your Man,http://changba.com/s/GZr_J5_BvVr4a-ftEiASmw,29,3,4,2,http://upscuw.changba.com/1208653739.mp3,2019-12-29 00:00:00,2019-12-29 00:00:00,2019-12-29 00:00:00,False
3,Nasa,http://changba.com/s/GZr_J5_BvVpGx_odmAqtBQ,25,0,3,0,http://upscuw.changba.com/1208653711.mp3,NaN,2019-12-29 00:00:00,2019-12-28 00:00:00,True
4,Scars To Your,http://changba.com/s/oTPTKN_3qXjKpIfapWDAAA,16,0,0,0,http://qiniuuwmp3.changba.com/1208653567.mp3,NaN,2019-12-29 00:00:00,2019-12-28 00:00:00,True
...,...,...,...,...,...,...,...,...,...,...,...
832,恒星流星,http://changba.com/s/lxnKkg0S0RVrwOn6rjxZOg,41,0,0,0,http://upuwmp3.changba.com/userdata/userwork/8...,2013-08-05 00:00:00,2013-08-02 00:00:00,2013-08-01 00:00:00,True
833,也许明天,http://changba.com/s/z1FbhFD6pZ669cvjWIcqfA,106,0,4,0,http://upuwmp3.changba.com/userdata/userwork/8...,NaN,2013-08-02 00:00:00,2013-08-01 00:00:00,True
834,剪爱,http://changba.com/s/TIO4hN908E8Wndka1ntQpQ,26,0,0,0,http://upuwmp3.changba.com/userdata/userwork/7...,2013-08-01 00:00:00,2013-08-01 00:00:00,2013-08-01 00:00:00,False
835,空白格,http://changba.com/s/co-YKmeP2LT5THi79AR9gQ,72,0,3,0,http://upuwmp3.changba.com/userdata/userwork/5...,2013-08-15 00:00:00,2013-08-01 00:00:00,2013-08-01 00:00:00,True


In [558]:
pd_merge.to_csv('changba.csv',encoding="utf-8-sig")